<a href="https://colab.research.google.com/github/ishika-gupta178/sign-language-detection/blob/main/Copy_of_project_sign_language_detection_through_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/dxli94/WLASL.git

Cloning into 'WLASL'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 228 (delta 24), reused 15 (delta 15), pack-reused 177
Receiving objects: 100% (228/228), 3.73 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [ ]:
import os
import json
import time
import sys
import urllib.request
from multiprocessing.dummy import Pool

import random

import logging
logging.basicConfig(filename='download_{}.log'.format(int(time.time())), filemode='w', level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))


def request_video(url, referer=''):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    headers = {'User-Agent': user_agent,
               }
    
    if referer:
        headers['Referer'] = referer

    request = urllib.request.Request(url, None, headers)  # The assembled request

    logging.info('Requesting {}'.format(url))
    response = urllib.request.urlopen(request)
    data = response.read()  # The data you need

    return data


def save_video(data, saveto):
    with open(saveto, 'wb+') as f:
        f.write(data)

    # please be nice to the host - take pauses and avoid spamming
    time.sleep(random.uniform(0.5, 1.5))


def download_youtube(url, dirname, video_id):
    raise NotImplementedError("Urllib cannot deal with YouTube links.")


def download_aslpro(url, dirname, video_id):
    saveto = os.path.join(dirname, '{}.swf'.format(video_id))
    if os.path.exists(saveto):
        logging.info('{} exists at {}'.format(video_id, saveto))
        return 

    data = request_video(url, referer='http://www.aslpro.com/cgi-bin/aslpro/aslpro.cgi')
    save_video(data, saveto)


def download_others(url, dirname, video_id):
    saveto = os.path.join(dirname, '{}.mp4'.format(video_id))
    if os.path.exists(saveto):
        logging.info('{} exists at {}'.format(video_id, saveto))
        return 
    
    data = request_video(url)
    save_video(data, saveto)


def select_download_method(url):
    if 'aslpro' in url:
        return download_aslpro
    elif 'youtube' in url or 'youtu.be' in url:
        return download_youtube
    else:
        return download_others


def download_nonyt_videos(indexfile, saveto='raw_videos'):
    content = json.load(open(indexfile))

    if not os.path.exists(saveto):
        os.mkdir(saveto)

    for entry in content:
        gloss = entry['gloss']
        instances = entry['instances']

        for inst in instances:
            video_url = inst['url']
            video_id = inst['video_id']
            
            logging.info('gloss: {}, video: {}.'.format(gloss, video_id))

            download_method = select_download_method(video_url)    
            
            if download_method == download_youtube:
                logging.warning('Skipping YouTube video {}'.format(video_id))
                continue

            try:
                download_method(video_url, saveto, video_id)
            except Exception as e:
                logging.error('Unsuccessful downloading - video {}'.format(video_id))


def check_youtube_dl_version():
    ver = os.popen('youtube-dl --version').read()

    assert ver, "youtube-dl cannot be found in PATH. Please verify your installation."
    assert ver >= '2020.03.08', "Please update youtube-dl to newest version."


def download_yt_videos(indexfile, saveto='raw_videos'):
    content = json.load(open(indexfile))
    
    if not os.path.exists(saveto):
        os.mkdir(saveto)
    
    for entry in content:
        gloss = entry['gloss']
        instances = entry['instances']

        for inst in instances:
            video_url = inst['url']
            video_id = inst['video_id']

            if 'youtube' not in video_url and 'youtu.be' not in video_url:
                continue

            if os.path.exists(os.path.join(saveto, video_url[-11:] + '.mp4')) or os.path.exists(os.path.join(saveto, video_url[-11:] + '.mkv')):
                logging.info('YouTube videos {} already exists.'.format(video_url))
                continue
            else:
                cmd = "youtube-dl \"{}\" -o \"{}%(id)s.%(ext)s\""
                cmd = cmd.format(video_url, saveto + os.path.sep)

                rv = os.system(cmd)
                
                if not rv:
                    logging.info('Finish downloading youtube video url {}'.format(video_url))
                else:
                    logging.error('Unsuccessful downloading - youtube video url {}'.format(video_url))

                # please be nice to the host - take pauses and avoid spamming
                time.sleep(random.uniform(1.0, 1.5))
    

if __name__ == '__main__':
    logging.info('Start downloading non-youtube videos.')
    download_nonyt_videos('/content/WLASL/start_kit/WLASL_v0.3.json')

    check_youtube_dl_version()
    logging.info('Start downloading youtube videos.')
    download_yt_videos('/content/WLASL/start_kit/WLASL_v0.3.json')



ERROR:root:Unsuccessful downloading - video 65225


Unsuccessful downloading - video 65225
Unsuccessful downloading - video 65225


Skipping YouTube video 68011
Skipping YouTube video 68011


Skipping YouTube video 68208
Skipping YouTube video 68208


Skipping YouTube video 68012
Skipping YouTube video 68012


...

Skipping YouTube video 39004
Skipping YouTube video 39004


ERROR:root:Unsuccessful downloading - video 38996


Unsuccessful downloading - video 38996
Unsuccessful downloading - video 38996


Skipping YouTube video 40114
Skipping YouTube video 40114


KeyboardInterrupt: ignored

In [ ]:
# preprocessing script for WLASL dataset
# 1. Convert .swf, .mkv file to mp4.
# 2. Extract YouTube frames and create video instances.

import os
import json
import cv2

import shutil

def convert_everything_to_mp4():
    cmd = 'bash /content/WLASL/start_kit/scripts/swf2mp4.sh'

    os.system(cmd)


def video_to_frames(video_path, size=None):
    """
    video_path -> str, path to video.
    size -> (int, int), width, height.
    """

    cap = cv2.VideoCapture(video_path)

    frames = []
    
    while True:
        ret, frame = cap.read()
    
        if ret:
            if size:
                frame = cv2.resize(frame, size)
            frames.append(frame)
        else:
            break

    cap.release()

    return frames


def convert_frames_to_video(frame_array, path_out, size, fps=25):
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


def extract_frame_as_video(src_video_path, start_frame, end_frame):
    frames = video_to_frames(src_video_path)

    return frames[start_frame: end_frame+1]


def extract_all_yt_instances(content):
    cnt = 1

    if not os.path.exists('videos'):
        os.mkdir('videos')

    for entry in content:
        instances = entry['instances']

        for inst in instances:
            url = inst['url']
            video_id = inst['video_id']

            if 'youtube' in url or 'youtu.be' in url:
                cnt += 1
                
                yt_identifier = url[-11:]

                src_video_path = os.path.join('raw_videos_mp4', yt_identifier + '.mp4')
                dst_video_path = os.path.join('videos', video_id + '.mp4')

                if not os.path.exists(src_video_path):
                    continue

                if os.path.exists(dst_video_path):
                    print('{} exists.'.format(dst_video_path))
                    continue

                # because the JSON file indexes from 1.
                start_frame = inst['frame_start'] - 1
                end_frame = inst['frame_end'] - 1

                if end_frame <= 0:
                    shutil.copyfile(src_video_path, dst_video_path)
                    continue

                selected_frames = extract_frame_as_video(src_video_path, start_frame, end_frame)
                
                # when OpenCV reads an image, it returns size in (h, w, c)
                # when OpenCV creates a writer, it requres size in (w, h).
                size = selected_frames[0].shape[:2][::-1]
                
                convert_frames_to_video(selected_frames, dst_video_path, size)

                print(cnt, dst_video_path)
            else:
                cnt += 1

                src_video_path = os.path.join('raw_videos_mp4', video_id + '.mp4')
                dst_video_path = os.path.join('videos', video_id + '.mp4')

                if os.path.exists(dst_video_path):
                    print('{} exists.'.format(dst_video_path))
                    continue

                if not os.path.exists(src_video_path):
                    continue

                print(cnt, dst_video_path)
                shutil.copyfile(src_video_path, dst_video_path)

        
def main():
    # 1. Convert .swf, .mkv file to mp4.
    convert_everything_to_mp4()

    content = json.load(open('/content/WLASL/start_kit/WLASL_v0.3.json'))
    extract_all_yt_instances(content)


if __name__ == "__main__":
    main()

2 videos/69241.mp4
12 videos/07069.mp4
19 videos/07068.mp4
24 videos/07070.mp4
26 videos/07099.mp4
31 videos/07074.mp4
42 videos/69302.mp4
53 videos/17710.mp4
58 videos/17733.mp4
59 videos/65540.mp4
60 videos/17734.mp4
61 videos/17711.mp4
62 videos/17712.mp4
63 videos/17713.mp4
69 videos/17709.mp4
72 videos/17720.mp4
73 videos/17721.mp4
74 videos/17722.mp4
75 videos/17723.mp4
76 videos/17724.mp4
80 videos/12328.mp4
83 videos/12312.mp4
89 videos/12311.mp4
91 videos/12338.mp4
92 videos/12313.mp4
93 videos/12314.mp4
94 videos/12315.mp4
95 videos/12316.mp4
96 videos/12317.mp4
97 videos/12318.mp4
98 videos/12319.mp4
99 videos/12320.mp4
105 videos/12326.mp4
106 videos/12327.mp4
112 videos/05728.mp4
115 videos/05749.mp4
116 videos/05750.mp4
117 videos/05729.mp4
118 videos/05730.mp4
120 videos/05731.mp4
121 videos/05732.mp4
122 videos/05733.mp4
123 videos/05734.mp4
126 videos/05727.mp4
128 videos/05739.mp4
129 videos/05740.mp4
130 videos/05741.mp4
131 videos/05742.mp4
132 videos/05743.mp4
140 

In [ ]:
!mkdir data

In [ ]:
!cp WLASL -r data/